# Classical Simulations: Water Splitting Catalyst Screening
## Complete DFT Workflow with Lithium Catalyst Model

**Team**: Alejandro Henriquez, Callum Follett,  Faisal Albarazi, Rubén Amórtegui

[github repo link](https://github.com/cafollet/electrolysis_catalyst_evaluation)

---

## Overview

This notebook demonstrates the **classical computational approach** to catalyst screening for water splitting using:
- **Density Functional Theory (DFT)** for quantum mechanical calculations
- **PBE Functional** for exchange-correlation approximation
- **PySCF** for molecular quantum chemistry

### Key Reactions
- **HER (Cathode)**: 2H⁺ + 2e⁻ → H₂  
- **OER (Anode)**: 2H₂O → O₂ + 4H⁺ + 4e⁻  

### Workflow

1. Define molecular geometries computationally  
2. DFT calculations for ground-state energies  
3. Calculations of adsorption energies  
4. Simple catalytic cycle / volcano interpretation  
5. Extract integrals for quantum computing  
6. TODO: Map integrals to qubits and run VQE  


## Installation & Imports


In [ ]:
%pip install -q pyscf pandas numpy

from pyscf import gto, scf, dft
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

print("✓ All imports successful")
print("✓ Ready to start classical simulations")


# STEP 1: Molecule Definition

We define all molecular species involved in water splitting:

| Species | Role | Bond Length | Spin State |
|---------|------|-------------|------------|
| H₂O | Reactant (substrate) | 0.96 Å | Singlet (0) |
| H₂ | HER product | 0.74 Å | Singlet (0) |
| O₂ | OER product | 1.21 Å | Triplet (2) |
| H | HER intermediate (H*) | - | Doublet (1) |
| O | OER intermediate (O*) | - | Triplet (2) |
| OH | OER intermediate (OH*) | 0.96 Å | Doublet (1) |
| Li₂ | Catalyst (clean) | 2.67 Å | Singlet (0) |
| Li₂-H | Adsorbate complex | - | Doublet (1) |

**Basis set:** STO-3G (minimal; for demo only).  
Real work: 6-31G*, def2-TZVP, etc.


In [ ]:
print("Defining molecular geometries...\n")

molecules = {}

# 1. H2O
molecules['H2O'] = gto.M(
    atom = [['O', (0.0000, 0.0000, 0.1173)],
            ['H', (0.0000, 0.7572, -0.4692)],
            ['H', (0.0000, -0.7572, -0.4692)]],
    basis = 'sto-3g',
    charge = 0,
    spin = 0,
)

# 2. H2
molecules['H2'] = gto.M(
    atom = [['H', (0.0, 0.0, 0.0)],
            ['H', (0.0, 0.0, 0.74)]],
    basis = 'sto-3g',
    charge = 0,
    spin = 0,
)

# 3. O2
molecules['O2'] = gto.M(
    atom = [['O', (0.0, 0.0, 0.0)],
            ['O', (0.0, 0.0, 1.21)]],
    basis = 'sto-3g',
    charge = 0,
    spin = 2,
)

# 4. H atom
molecules['H'] = gto.M(
    atom = [['H', (0, 0, 0)]],
    basis = 'sto-3g',
    charge = 0,
    spin = 1,
)

# 5. O atom
molecules['O'] = gto.M(
    atom = [['O', (0, 0, 0)]],
    basis = 'sto-3g',
    charge = 0,
    spin = 2,
)

# 6. OH radical
molecules['OH'] = gto.M(
    atom = [['O', (0.0, 0.0, 0.0)],
            ['H', (0.0, 0.96, 0.0)]],
    basis = 'sto-3g',
    charge = 0,
    spin = 1,
)

# 7. Li2 catalyst
molecules['Li2'] = gto.M(
    atom = [['Li', (0, 0, 0)],
            ['Li', (0, 0, 2.67)]],
    basis = 'sto-3g',
    charge = 0,
    spin = 0,
)

# 8. Li2-H adsorbate
molecules['Li2-H'] = gto.M(
    atom = [['Li', (0, 0, 0)],
            ['H',  (0, 0, 1.6)],
            ['Li', (0, 0, 2.67)]],
    basis = 'sto-3g',
    charge = 0,
    spin = 1,
)

rows = []
for name, mol in molecules.items():
    rows.append({
        'Species': name,
        'Atoms': mol.natm,
        'Electrons': mol.nelectron,
        'Basis functions': mol.nao_nr(),
    })

df_mol = pd.DataFrame(rows)
df_mol


# STEP 2: Ground-State Energy Calculations (DFT)

We compute ground-state electronic energies with **PBE DFT**.


In [ ]:
energies = {}

def e_ev(E):
    return E * 27.211

print('Running DFT (PBE) calculations...\n')

# H2O
mf = dft.RKS(molecules['H2O']); mf.xc = 'PBE'; mf.verbose = 0
energies['H2O'] = mf.kernel()

# H2
mf = dft.RKS(molecules['H2']); mf.xc = 'PBE'; mf.verbose = 0
energies['H2'] = mf.kernel()

# O2
mf = dft.UKS(molecules['O2']); mf.xc = 'PBE'; mf.verbose = 0
energies['O2'] = mf.kernel()

# H
mf = scf.UHF(molecules['H']); mf.verbose = 0
energies['H'] = mf.kernel()

# O
mf = scf.UHF(molecules['O']); mf.verbose = 0
energies['O'] = mf.kernel()

# OH
mf = scf.UHF(molecules['OH']); mf.verbose = 0
energies['OH'] = mf.kernel()

# Li2
mf = scf.RHF(molecules['Li2']); mf.verbose = 0
energies['Li2'] = mf.kernel()

# Li2-H
mf = scf.UHF(molecules['Li2-H']); mf.verbose = 0
energies['Li2-H'] = mf.kernel()

energy_rows = []
for k in ['H2O','H2','O2','H','O','OH','Li2','Li2-H']:
    E = energies[k]
    energy_rows.append({
        'Species': k,
        'Energy (Hartree)': f'{E: .6f}',
        'Energy (eV)': f'{e_ev(E): .4f}',
    })

df_energies = pd.DataFrame(energy_rows)
df_energies


# STEP 3: Adsorption & Reaction Energies


In [ ]:
# H adsorption on Li2: Li2 + H2 -> Li2-H
delta_E_H = energies['Li2-H'] - energies['Li2'] - energies['H2']

# Overall water splitting: H2O -> H2 + 1/2 O2
delta_E_reaction = energies['H2'] + 0.5 * energies['O2'] - energies['H2O']

print('H Adsorption on Li₂ (HER key intermediate)\n')
print(f'ΔE_H (Hartree) = {delta_E_H: .6f}')
print(f'ΔE_H (eV)      = {e_ev(delta_E_H): .4f}\n')

print('Water Splitting: H₂O → H₂ + ½O₂\n')
print(f'ΔE_reaction (Hartree) = {delta_E_reaction: .6f}')
print(f'ΔE_reaction (eV)      = {e_ev(delta_E_reaction): .4f}')


In [ ]:
print('Assessment of H adsorption (volcano concept):\n')

dEh = e_ev(delta_E_H)

if dEh < 0:
    print(f'• H binds (ΔE_H = {dEh:.3f} eV)')
    if -0.3 < dEh < 0:
        print('• In or near optimal HER range (-0.3, 0) eV')
    elif dEh <= -0.3:
        print('• Binding too strong, H might not desorb easily')
else:
    print(f'• H does NOT bind (ΔE_H = {dEh:.3f} eV) – bad HER catalyst')

print('\nAssessment of water splitting energetics:\n')
dEr = e_ev(delta_E_reaction)
if dEr > 0:
    print(f'• Reaction is ENDOTHERMIC: requires ≈ {dEr:.2f} eV per H₂O')
    print('• This is why external voltage (electrolysis) is needed.')
else:
    print('• Reaction would be exothermic (unphysical for water splitting).')


# STEP 5: Extract Integrals for Quantum Computing

We extract one- and two-electron integrals (h1e, h2e) and overlap matrices S for H₂O, H₂, and Li₂-H.


In [ ]:
integrals = {}

print('Extracting one- and two-electron integrals for key systems...\n')

def extract_integrals(label, mol, method='RKS'):
    if method == 'RKS':
        mf = dft.RKS(mol); mf.xc = 'PBE'; mf.verbose = 0
    elif method == 'UKS':
        mf = dft.UKS(mol); mf.xc = 'PBE'; mf.verbose = 0
    elif method == 'RHF':
        mf = scf.RHF(mol); mf.verbose = 0
    else:
        mf = scf.UHF(mol); mf.verbose = 0
    mf.kernel()
    h1e = mf.get_hcore()
    h2e = mol.intor('int2e')
    S   = mf.get_ovlp()
    integrals[label] = {
        'h1e': h1e,
        'h2e': h2e,
        'S': S,
        'nmo': mol.nao_nr(),
        'nelectron': mol.nelectron,
        'nuclear_rep': mf.energy_nuc(),
    }
    print(label + ':', 'h1e', h1e.shape, ', h2e', h2e.shape, ', S', S.shape, ', nmo=', mol.nao_nr(), ', nelec=', mol.nelectron)

extract_integrals('H2O', molecules['H2O'], method='RKS')
extract_integrals('H2',  molecules['H2'],  method='RKS')
extract_integrals('Li2-H', molecules['Li2-H'], method='UHF')


# Quantum Computing Phase

Now that we understand how its done classically, the next steps are to build the pipeline. This pipeline will:

1. **Build Molecular Hamiltonian**  
2. **Map to Qubits** (Jordan–Wigner, Bravyi–Kitaev, Parity)
3. **Run VQE or SQD** to obtain improved energies


In [1]:
import time
import jax
import simulation
from simulation import Chemical, test_hydrogen, test_hydrogen_sqd, test_li_ion, test_li_ion_sqd, test_aluminum
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

jax.config.update("jax_enable_x64", True)

# Testing time to run VQE vs SQD
time_start = time.time()
h_nrg = test_hydrogen()
vqe_time_h = time.time() - time_start


Optimizing Hydrogen Molecule, jordan_wigner basis chosen
Step   0 | Energy = -1.13708098 | params = [0.19999999 0.         0.        ]
Step  40 | Energy = -1.13730597 | params = [0.22393922 0.         0.        ]
Step  80 | Energy = -1.13730603 | params = [0.22370866 0.         0.        ]
Step 120 | Energy = -1.13730604 | params = [0.22361696 0.         0.        ]
Step 160 | Energy = -1.13730605 | params = [0.22350536 0.         0.        ]
Estimated ground state energy (VQE): -1.137306051209999


In [5]:
time_start = time.time()
h_nrg_2 = test_hydrogen_sqd()
sqd_time_h = time.time() - time_start

Running SQD
converged SCF energy = -0.945015061285214
E(CCSD) = -1.017584403104664  E_corr = -0.07256934181945014


qiskit_runtime_service._discover_account:WARNING:2025-12-05 11:28:15,422: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2025-12-05 11:28:17,976: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: Compute for Python. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-12-05 11:28:18,590: Loading instance: Compute for Python, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-05 11:28:20,842: Using instance: Compute for Python, plan: open


Quantum Portion Finished
Starting Post-Processing
Iteration 1
	Subsample 0
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 1
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 2
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 3
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 4
		Energy: -1.017584403020935
		Subspace dimension: 4
Iteration 2
	Subsample 0
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 1
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 2
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 3
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 4
		Energy: -1.017584403020935
		Subspace dimension: 4


In [6]:
time_start = time.time()
li_ion_nrg = test_li_ion()
vqe_time_li_ion = time.time() - time_start

Optimizing ['Li'], jordan_wigner basis chosen
Step   0 | Energy = -6.91850504 | params = [ 0.19999993  0.          0.          0.          0.          0.
  0.          0.19999979  0.          0.          0.          0.
  0.19999979  0.          0.          0.19999979 -0.02665254  0.
  0.          0.         -0.02665254  0.          0.          0.        ]
Step  10 | Energy = -7.11047975 | params = [0.06465094 0.         0.         0.         0.         0.
 0.         0.069901   0.         0.         0.         0.
 0.06989929 0.         0.         0.06989759 0.04739642 0.
 0.         0.         0.04739642 0.         0.         0.        ]
Step  20 | Energy = -7.11739531 | params = [0.06432129 0.         0.         0.         0.         0.
 0.         0.06015809 0.         0.         0.         0.
 0.06015809 0.         0.         0.06015809 0.01891586 0.
 0.         0.         0.01891586 0.         0.         0.        ]
Step  30 | Energy = -7.13107178 | params = [4.69938987e-02 0.00000

In [7]:
time_start = time.time()
li_ion_nrg_2 = test_li_ion_sqd()
sqd_time_li_ion = time.time() - time_start

Running SQD
converged SCF energy = -7.13544762901411
E(CCSD) = -7.135654284249094  E_corr = -0.00020665523498186
Quantum Portion Finished
Starting Post-Processing
Iteration 1
	Subsample 0
		Energy: -7.135447629014113
		Subspace dimension: 1
	Subsample 1
		Energy: -7.135447629014113
		Subspace dimension: 1
	Subsample 2
		Energy: -7.135447629014113
		Subspace dimension: 1
	Subsample 3
		Energy: -7.135447629014113
		Subspace dimension: 1
	Subsample 4
		Energy: -7.135447629014113
		Subspace dimension: 1
Iteration 2
	Subsample 0
		Energy: -7.135447629014113
		Subspace dimension: 1
	Subsample 1
		Energy: -7.135447629014113
		Subspace dimension: 1
	Subsample 2
		Energy: -7.135447629014113
		Subspace dimension: 1
	Subsample 3
		Energy: -7.135447629014113
		Subspace dimension: 1
	Subsample 4
		Energy: -7.135447629014113
		Subspace dimension: 1


In [8]:
print(f"Times to compile - Energy: ")
print(f"\nHydrogen")
print(f"VQE: {round(vqe_time_h, 2)}s - {h_nrg} H, "
      f"SQD: {round(sqd_time_h, 2)}s - {min(h_nrg_2)} H")
print(f"\nLithium")
print(f"VQE time: {round(vqe_time_li_ion, 2)}s - {li_ion_nrg} H, "
      f"SQD Time: {round(sqd_time_li_ion, 2)}s - {min(li_ion_nrg_2)} H")
print(f"\nSQD Average accuracy: {round(((min(li_ion_nrg_2)/li_ion_nrg)+(min(h_nrg_2)/h_nrg))*50, 0)}%")
print(f"\nSQD Li ion speedup: {round(vqe_time_li_ion/sqd_time_li_ion, 0)}x")

Times to compile - Energy: 

Hydrogen
VQE: 16.81s - -1.137306051209999 H, SQD: 14.18s - -1.017584403020935 H

Lithium
VQE time: 234.2s - -7.135459935180906 H, SQD Time: 0.47s - -7.135447629014113 H

SQD Average accuracy: 95.0%

SQD Li ion speedup: 501.0x


In [9]:
h_nrg = test_hydrogen_sqd()
print("Hydrogen: ", min(h_nrg), "H")

al_nrg = test_aluminum()
print("Aluminum: ", min(al_nrg), "H")

Running SQD
converged SCF energy = -0.945015061285214
E(CCSD) = -1.017584403104664  E_corr = -0.07256934181945014


qiskit_runtime_service._discover_account:WARNING:2025-12-05 11:34:26,931: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2025-12-05 11:34:29,799: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: Compute for Python. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-12-05 11:34:30,312: Loading instance: Compute for Python, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-05 11:34:32,360: Using instance: Compute for Python, plan: open


Quantum Portion Finished
Starting Post-Processing
Iteration 1
	Subsample 0
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 1
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 2
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 3
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 4
		Energy: -1.017584403020935
		Subspace dimension: 4
Iteration 2
	Subsample 0
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 1
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 2
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 3
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 4
		Energy: -1.017584403020935
		Subspace dimension: 4
Hydrogen:  -1.017584403020935 H
Running SQD
converged SCF energy = -238.858362036882  <S^2> = 0.75000313  2S+1 = 2.0000031
E(UCCSD) = -238.8839189685205  E_corr = -0.02555693163899903


qiskit_runtime_service._discover_account:WARNING:2025-12-05 11:34:44,070: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2025-12-05 11:34:46,369: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: Compute for Python. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-12-05 11:34:46,982: Loading instance: Compute for Python, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-05 11:34:49,337: Using instance: Compute for Python, plan: open


Quantum Portion Finished
Starting Post-Processing
Iteration 1
	Subsample 0
		Energy: -238.88365718145116
		Subspace dimension: 24
	Subsample 1
		Energy: -238.88365718145116
		Subspace dimension: 24
	Subsample 2
		Energy: -238.88365718145116
		Subspace dimension: 24
	Subsample 3
		Energy: -238.88365718145116
		Subspace dimension: 24
	Subsample 4
		Energy: -238.88365718145116
		Subspace dimension: 24
Iteration 2
	Subsample 0
		Energy: -238.88365718145116
		Subspace dimension: 24
	Subsample 1
		Energy: -238.88365718145116
		Subspace dimension: 24
	Subsample 2
		Energy: -238.88365718145116
		Subspace dimension: 24
	Subsample 3
		Energy: -238.88365718145116
		Subspace dimension: 24
	Subsample 4
		Energy: -238.88365718145116
		Subspace dimension: 24
Aluminum:  -238.88365718145116 H


Testing a more complex atom:

In [10]:
import pyscf

stepsize = 50
num_steps = 100

nickel = pyscf.gto.Mole()
nickel.build(
    atom=[["Ni", (0, 0, 0)]],
    basis="def2-SVP",
)
energy = Chemical(nickel, name="Nickel Atom").run_simulation(stepsize, num_steps, method="sqd")

Running SQD
converged SCF energy = -1506.50503479897
E(CCSD) = -1506.835268531078  E_corr = -0.3302337321050073


qiskit_runtime_service._discover_account:WARNING:2025-12-05 11:36:19,176: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2025-12-05 11:36:21,585: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: Compute for Python. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-12-05 11:36:22,096: Loading instance: Compute for Python, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-05 11:36:24,349: Using instance: Compute for Python, plan: open


Quantum Portion Finished
Starting Post-Processing
Iteration 1
	Subsample 0
		Energy: -1481.080094389693
		Subspace dimension: 1521
	Subsample 1
		Energy: -1481.080094389693
		Subspace dimension: 1521
	Subsample 2
		Energy: -1481.080094389693
		Subspace dimension: 1521
	Subsample 3
		Energy: -1481.080094389693
		Subspace dimension: 1521
	Subsample 4
		Energy: -1481.080094389693
		Subspace dimension: 1521
Iteration 2
	Subsample 0
		Energy: -1501.6720806760732
		Subspace dimension: 2601
	Subsample 1
		Energy: -1489.329067837928
		Subspace dimension: 2601
	Subsample 2
		Energy: -1487.7153657143656
		Subspace dimension: 2601
	Subsample 3
		Energy: -1497.3274462900804
		Subspace dimension: 2601
	Subsample 4
		Energy: -1493.6824208430787
		Subspace dimension: 2550
Iteration 3
	Subsample 0
		Energy: -1501.7998309773761
		Subspace dimension: 2601
	Subsample 1
		Energy: -1502.9492035959036
		Subspace dimension: 2601
	Subsample 2
		Energy: -1501.6720838390324
		Subspace dimension: 2601
	Subsample

In [11]:
print("Found Nickel Ground State Energy: ", min(energy))

Found Nickel Ground State Energy:  -1505.9349941133014


Now for testing the calculation of adsorption energy: $2\text{Al} + 3\text{H}_2 \to 2\text{AlH}_3$

In [12]:
from simulation import adsorption_energy

stepsize = 50
num_steps = 100

hydrogen = pyscf.gto.Mole()
hydrogen.build(
    atom=[["H", (0, 0, -0.69434785)], ["H", (0, 0, 0.69434785)]],
    basis="sto-3g",
)

aluminum = pyscf.gto.Mole()
aluminum.build(
    atom=[["Al", (0, 0, 0)]],
    basis="sto-3g",
    spin = 1,
)

al_h3 = pyscf.gto.Mole()
al_h3.build(
    atom=[["Al", (0, 0, 0)], ["H", (0, 2.682, 0)], ["H", (2.3229, -1.3411, 0)], ["H", (-2.3229, -1.3411, 0)]],
    basis="sto-3g",
)


print("Adsorption energy of Arbitrary AlH_3 intermediate reaction:", adsorption_energy(stepsize, num_steps, hydrogen, aluminum, al_h3))

Running SQD
converged SCF energy = -0.945015061285214
E(CCSD) = -1.017584403104664  E_corr = -0.07256934181945014


qiskit_runtime_service._discover_account:WARNING:2025-12-05 11:43:15,035: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2025-12-05 11:43:17,950: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: Compute for Python. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-12-05 11:43:18,564: Loading instance: Compute for Python, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-05 11:43:20,508: Using instance: Compute for Python, plan: open


Quantum Portion Finished
Starting Post-Processing
Iteration 1
	Subsample 0
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 1
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 2
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 3
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 4
		Energy: -1.017584403020935
		Subspace dimension: 4
Iteration 2
	Subsample 0
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 1
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 2
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 3
		Energy: -1.017584403020935
		Subspace dimension: 4
	Subsample 4
		Energy: -1.017584403020935
		Subspace dimension: 4
Running SQD
converged SCF energy = -238.858362036882  <S^2> = 0.75000313  2S+1 = 2.0000031
E(UCCSD) = -238.8839189685205  E_corr = -0.02555693163899903


qiskit_runtime_service._discover_account:WARNING:2025-12-05 11:43:30,925: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2025-12-05 11:43:33,514: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: Compute for Python. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-12-05 11:43:34,026: Loading instance: Compute for Python, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-05 11:43:36,484: Using instance: Compute for Python, plan: open


Quantum Portion Finished
Starting Post-Processing
Iteration 1
	Subsample 0
		Energy: -238.88365718145116
		Subspace dimension: 24
	Subsample 1
		Energy: -238.88365718145116
		Subspace dimension: 24
	Subsample 2
		Energy: -238.88365718145116
		Subspace dimension: 24
	Subsample 3
		Energy: -238.88365718145116
		Subspace dimension: 24
	Subsample 4
		Energy: -238.88365718145116
		Subspace dimension: 24
Iteration 2
	Subsample 0
		Energy: -238.88365718145116
		Subspace dimension: 24
	Subsample 1
		Energy: -238.88365718145116
		Subspace dimension: 24
	Subsample 2
		Energy: -238.88365718145116
		Subspace dimension: 24
	Subsample 3
		Energy: -238.88365718145116
		Subspace dimension: 24
	Subsample 4
		Energy: -238.88365718145116
		Subspace dimension: 24
Running SQD
converged SCF energy = -240.096973738954
E(CCSD) = -240.3404671981042  E_corr = -0.2434934591502327


qiskit_runtime_service._discover_account:WARNING:2025-12-05 11:43:45,393: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2025-12-05 11:43:47,955: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: Compute for Python. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-12-05 11:43:48,569: Loading instance: Compute for Python, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-05 11:43:50,615: Using instance: Compute for Python, plan: open


Quantum Portion Finished
Starting Post-Processing
Iteration 1
	Subsample 0
		Energy: -240.3092251848027
		Subspace dimension: 442
	Subsample 1
		Energy: -240.29642523448908
		Subspace dimension: 484
	Subsample 2
		Energy: -240.33608507484362
		Subspace dimension: 550
	Subsample 3
		Energy: -240.29815985986986
		Subspace dimension: 462
	Subsample 4
		Energy: -240.29432529672738
		Subspace dimension: 437
Iteration 2
	Subsample 0
		Energy: -240.34764112368973
		Subspace dimension: 812
	Subsample 1
		Energy: -240.35615371622245
		Subspace dimension: 728
	Subsample 2
		Energy: -240.34571170097138
		Subspace dimension: 675
	Subsample 3
		Energy: -240.35750322798458
		Subspace dimension: 780
	Subsample 4
		Energy: -240.35029691526157
		Subspace dimension: 780
Iteration 3
	Subsample 0
		Energy: -240.36190826036832
		Subspace dimension: 899
	Subsample 1
		Energy: -240.36313410219125
		Subspace dimension: 960
	Subsample 2
		Energy: -240.3595192807876
		Subspace dimension: 930
	Subsample 3
		Ener

We see that for most single-atom molecules, or otherwise simple molecules, the SQD provides a ground state of around 95-99% accuracy to the classical CCSD simulation method, however as the molecules get more complex (AlH_3, for example), the quantum-run simulation provides a ground state energy lower than the aforementioned classical CCSD calculation!
($-240.367\text{H} < -240.340\text{H}$ by $0.02$!)